In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
!python --version

Python 3.10.12


## Connect with Google Drive

In [2]:
# once you mount your google drive, you can read data from your google drive into the colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_dir = '/content/drive/MyDrive/GenAI_Course_Master/'
!ls $file_dir

OpenAI_API_Key_1.txt


In [5]:
filepath = f"{file_dir}/OpenAI_API_Key_1.txt"
filepath

'/content/drive/MyDrive/GenAI_Course_Master//OpenAI_API_Key_1.txt'

In [6]:
# import openai and set the API key
import openai

api_key = ""
with open(filepath, "r") as f:
  api_key = ' '.join(f.readlines())

# Read Data

In [7]:
import pandas as pd
df = pd.read_csv('laptop_data.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


# Stage 1: Intent Clarity and Intent Confirmation

## Initialize Conversation

3 Types of user in conversation
 - System
 - User
 - Assistant

In [8]:
def initialize_conversation():
    '''
    Returns a list of messages [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'low','Multitasking': 'high','Processing speed': 'high','Budget': '150000'}

    system_message = f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [9]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The values for all keys, except \'budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user.\n    The value for \'budget\' should be a numerical value extracted from the user\'s r

In [10]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [11]:
def get_chat_model_completions(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        # max_tokens = 3000
    )
    return response.choices[0].message.content

In [12]:
debug_introduction = get_chat_model_completions(debug_conversation)

In [13]:
debug_introduction

'Assistant: "Hi there! I\'m an intelligent laptop gadget expert and I\'m here to help you find the best laptop for your needs. Please tell me a little bit about yourself and what you\'ll be using the laptop for."'

## Start User conversation

In [14]:
debug_user_input = "Hi, I am Anand. I need a laptop for coding."

In [15]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great! As a coder, you likely require a laptop that can handle multitasking and has a good processing speed. May I know what kind of coding work do you primarily focus on? Are you more involved in web development, data analysis, or software development? Understanding the specific type of coding work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.


## Moderation Check

In [16]:
def moderation_check(user_input):
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]
    if moderation_output.flagged == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [17]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [18]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("I need a laptop to hack NASA."))

Flagged
Not Flagged


## Intent Confirmation Layer

In [19]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number with currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """
    messages = [{"role": "user", "content": prompt}]

    confirmation = get_chat_model_completions(messages)
    return confirmation

In [20]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)

No


## Dictionary Present

In [21]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:

{{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}}

Please note that the values for 'Display quality' and 'Portability' are set to 'low' since they are not a priority for your coding tasks.

Is there anything else I can assist you with?
"""

In [22]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}

            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}

            Here is the input {response}

            """
    messages = [{"role": "system", "content": prompt}]
    confirmation = get_chat_model_completions(messages)
    return confirmation

In [23]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}


# Stage 2: Product Mapping and Information Extraction

## Product Mapping

In [24]:
def product_map_layer(laptop_description):

  delimiter = "#####"
  lap_spec = {
      "GPU intensity":"(Type of the Graphics Processor)",
      "Display quality":"(Display Type, Screen Resolution, Display Size)",
      "Portability":"(Laptop Weight)",
      "Multitasking":"(RAM Size)",
      "Processing speed":"(CPU Type, Core, Clock Speed)"
  }

  values = {'low','medium','high'}

  prompt=f"""
  You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
  To analyze each laptop, perform the following steps:
  Step 1: Extract the laptop's primary features from the description {laptop_description}
  Step 2: Store the extracted features in {lap_spec} \
  Step 3: Classify each of the items in {lap_spec}into {values} based on the following rules: \
  {delimiter}
  GPU Intensity: low: < for Integrated graphics or entry-level dedicated graphics like Intel UHD > , \n
  medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
  high: < High-end dedicated graphics like Nvidia > , \n

  Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
  medium: < if Full HD resolution (1920x1080) or higher. > , \n
  high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n

  Portability: low: < if laptop weight is less than 1.51 kg > , \n
  medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
  high: <if laptop weight is greater than 2.51 kg> \n

  Multitasking: low: < If RAM size is 8GB, 12GB > , \n
  medium: < if RAM size is 16GB > , \n
  high: < if RAM size is 32GB, 64GB> \n

  Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3> , \n
  medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5 > , \n
  high: < if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher > \n
  {delimiter}

  {delimiter}
  Here are some input output pairs for few-shot learning:
  input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
  output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

  input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for multitasking and productivity. With 16GB of RAM and an SSD, it offers fast and efficient performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for decent graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. Priced at 130,000, it offers top-notch performance and portability for professionals."
  output2: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'high', 'Processing speed':'high'}}

  input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an Intel Core i9 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an AMD Radeon graphics card, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it is relatively lightweight for its size. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals. Priced at 280,000, it caters to users who require uncompromising performance and a superior display for their demanding workloads."
  output3: {{'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
  {delimiter}

  Follow the above instructions step-by-step and output the dictionary {lap_spec} for the following laptop {laptop_description}.
  """

  #see that we are using the Completion endpoint and not the Chatcompletion endpoint
  messages = [{"role": "system", "content": prompt}]
  response = get_chat_model_completions(messages)
  return response

In [26]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data.csv')

laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [27]:
laptop_df['Description'][0]

'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.'

In [28]:
## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

In [29]:
laptop_df['laptop_feature'][0]

"{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}"

In [30]:
laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'GPU intensity': 'medium', 'Display quality':..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'GPU intensity': 'high', 'Display quality': '..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'GPU intensity': 'medium', 'Display quality':..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'GPU intensity': 'low', 'Display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'GPU intensity': 'high', 'Display quality': '..."


In [31]:
laptop_df.to_csv("updated_laptop.csv", index=False,header = True)

In [32]:
!ls

drive  laptop_data.csv	sample_data  updated_laptop.csv


## Extract dictionary from string

In [33]:
import ast

string_representation = "{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}"

# Convert the string to a dictionary using ast.literal_eval
dictionary_representation = ast.literal_eval(string_representation)

print(dictionary_representation)


{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}


In [34]:
type(dictionary_representation)

dict

In [35]:
import re
import ast

def extract_dictionary_from_string(string: str) -> dict:
  # Use regular expression to extract exact dictionary part
  regex_pattern = r"\{[^{}]+\}"
  dictionary_matches = re.findall(regex_pattern, string)

  if dictionary_matches:
    dictionary_match = dictionary_matches[0]
    dictionary_string = dictionary_match.lower()

    # Convert dictionary string into a python dictionary
    dictionary = ast.literal_eval(dictionary_string)
  return dictionary

In [36]:
string = "{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'medium'}"
dictionary = extract_dictionary_from_string(string)

In [37]:
dictionary

{'gpu intensity': 'medium',
 'display quality': 'medium',
 'portability': 'medium',
 'multitasking': 'high',
 'processing speed': 'medium'}

In [38]:
type(dictionary)

dict

In [39]:
response_dict_n = "{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'low', 'Budget': '90000'}"

In [40]:
def compare_laptop_with_users(user_req_str):
  # Convert user req string into python dictionary
  user_req = extract_dictionary_from_string(user_req_str)
  user_budget = user_req.get("budget", '0')
  user_budget = user_budget.replace(",", "").split(" ")[0]
  user_budget = int(user_budget)
  print("user_req : ", user_req)
  print("user_budget : ", user_budget)

  # Load laptop dataframe, and filter laptops whose price is less than or equal to user_budget
  laptop_df= pd.read_csv('updated_laptop.csv')
  filtered_laptops = laptop_df.copy()
  filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(",", "").astype(int)
  filtered_laptops = filtered_laptops[filtered_laptops.Price <= user_budget].copy()

  mappings = {'low': 0, 'medium': 1, 'high': 2}

  # Create Score column in dataframe and initialize it to 0
  filtered_laptops['Score'] = 0
  # Calculate score based on user_req
  for index, row in filtered_laptops.iterrows():
    user_product_match_str = row['laptop_feature']
    laptop_values = extract_dictionary_from_string(user_product_match_str)
    print("laptop_values : ", laptop_values)
    score = 0

    for key, user_value in user_req.items():
      if key.lower() == 'budget':
        continue # Skip budget comparision, its buisness requirement

      laptop_value = laptop_values.get(key, None)
      print(f"key : {key}, laptop_value : {laptop_value}")

      laptop_mappings = mappings.get(laptop_value.lower(), -1)
      user_mappings = mappings.get(user_value, -1)

      # If the laptop value is greater than or equal to the user value then increment score by 1
      if laptop_mappings >= user_mappings:
        score += 1

    filtered_laptops.loc[index, 'Score'] = score

  # Sort the laptops by score in descending order and return top 3 products
  top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
  top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
  print("top_laptops :", top_laptops)

  return top_laptops.to_json(orient='records')

In [41]:
top_3_laptops = compare_laptop_with_users(response_dict_n)
print(top_3_laptops)

user_req :  {'gpu intensity': 'low', 'display quality': 'high', 'portability': 'low', 'multitasking': 'high', 'processing speed': 'low', 'budget': '90000'}
user_budget :  90000
laptop_values :  {'gpu intensity': 'medium', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'high', 'processing speed': 'medium'}
key : gpu intensity, laptop_value : medium
key : display quality, laptop_value : medium
key : portability, laptop_value : medium
key : multitasking, laptop_value : high
key : processing speed, laptop_value : medium
laptop_values :  {'gpu intensity': 'high', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'high', 'processing speed': 'high'}
key : gpu intensity, laptop_value : high
key : display quality, laptop_value : medium
key : portability, laptop_value : medium
key : multitasking, laptop_value : high
key : processing speed, laptop_value : high
laptop_values :  {'gpu intensity': 'medium', 'display quality': 'medium', 'portability': 'high'

## Validate recomendations are good enough for user's requirement

In [42]:
import json

def recommendation_validation(laptop_recommendation):
  data = json.loads(laptop_recommendation)
  data1 = []
  for i in range(len(data)):
    if data[i]['Score'] > 2:
      data1.append(data[i])
  return data1

In [43]:
validated_data = recommendation_validation(top_3_laptops)
print(validated_data)

[{'Brand': 'Dell', 'Model Name': 'Inspiron', 'Core': 'i5', 'CPU Manufacturer': 'Intel', 'Clock Speed': '2.4 GHz', 'RAM Size': '8GB', 'Storage Type': 'SSD', 'Display Type': 'LCD', 'Display Size': '15.6"', 'Graphics Processor': 'Intel UHD', 'Screen Resolution': '1920x1080', 'OS': 'Windows 10', 'Laptop Weight': '2.5 kg', 'Special Features': 'Backlit Keyboard', 'Warranty': '1 year', 'Average Battery Life': '6 hours', 'Price': 35000, 'Description': 'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU 

In [44]:
len(validated_data)

3

# Stage 3: Product Reccomendation Layer

In [45]:
def initialize_conversation_for_product_recommendation(products):
  system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to solve the user \
    queries about any product from the catalogue: {products}. \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name>: <Majof specification of laptop>, <Price in Rs>
    2. <Laptop Name>: <Majof specification of laptop>, <Price in Rs>

  """
  conversation = [{"role": "system", "content": system_message}]
  return conversation

In [46]:
debug_conversation_record = initialize_conversation_for_product_recommendation(top_3_laptops)

In [47]:
debug_recommendation = get_chat_model_completions(debug_conversation_record)

In [48]:
print(debug_recommendation)

1. HP EliteBook: Premium laptop designed for professionals, Intel Core i7 processor, 16GB RAM, SSD storage, 14" LED display, Windows 11 OS, Fingerprint Sensor, 3-year warranty, 8 hours battery life. Price: Rs 90,000.

2. MSI GL65: High-performance gaming laptop, Intel Core i7 processor, 16GB RAM, HDD+SSD storage, 15.6" IPS display, NVIDIA GTX graphics, Windows 10 OS, RGB Keyboard, 2-year warranty, 4 hours battery life. Price: Rs 55,000.

3. Dell Inspiron: Versatile laptop with powerful performance and affordability, Intel Core i5 processor, 8GB RAM, SSD storage, 15.6" LCD display, Intel UHD graphics, Windows 10 OS, Backlit Keyboard, 1-year warranty, 6 hours battery life. Price: Rs 35,000.


In [49]:
debug_conversation_record.append({"role": "user", "content": f"This is my user profile: {response_dict_n}"})

In [50]:
debug_conversation_record.append({"role": "assistant", "content": debug_recommendation})

In [51]:
debug_user_input = "which is ideal for travel ?"

In [52]:
debug_conversation_record.append({"role": "user", "content": debug_user_input})

In [53]:
debug_response_ass_reco = get_chat_model_completions(debug_conversation_record)

In [54]:
debug_response_ass_reco

'Based on your user profile, where portability is listed as "low", the Dell Inspiron would be the most suitable option for travel. It weighs just 2.5 kg, making it highly portable and convenient to carry around. Additionally, it has a battery life of up to 6 hours, allowing you to use it for an extended period without needing to charge frequently.'

# Integrate Stage 1, 2 & 3

In [74]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_3_laptops = None
    user_input = ''


    while "exit" not in user_input:
        user_input = input("")


        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break


        if top_3_laptops is None:
            conversation.append({"role": "user", "content": user_input})


            response_assistant = get_chat_model_completions(conversation)


            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)


            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break


            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)


                moderation = moderation_check(response)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break




                print('\n' + response + '\n')
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptop_with_users(response)


                validated_reco = recommendation_validation(top_3_laptops)


                if len(validated_reco) == 0:
                    print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                    break


                conversation_reco = initialize_conversation_for_product_recommendation(validated_reco)
                recommendation = get_chat_model_completions(conversation_reco)


                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break


                conversation_reco.append({"role": "user", "content": "This is my user profile" + response})


                conversation_reco.append({"role": "assistant", "content": recommendation})


                print(recommendation + '\n')


        else:
            conversation_reco.append({"role": "user", "content": user_input})


            response_asst_reco = get_chat_model_completions(conversation_reco)


            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break


            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [75]:
dialogue_mgmt_system()


Assistant: "Hi there! I'm an intelligent laptop gadget expert and I'm here to help you find the best laptop for your needs. Please tell me a little bit about yourself and what you'll be using the laptop for."

I need laptop for backend development, GPU is not required for me, 

Great! As a backend developer, you primarily focus on coding and server-side tasks, which means you don't require a high GPU intensity. However, you would still need a laptop with good processing speed and multitasking capability. Display quality and portability may not be as important for your work. Could you please let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements.


No

80 k

Thank you for providing your budget. Based on your requirements as a backend developer, I would suggest the following values for the different keys:

'GPU intensity': 'low'
'Display quality': 'medium'
'Portability': 'medium'
'Multitasking': 'hi